In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100



In [94]:
# Load Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

player_injuries_df = pd.read_csv('player_injuries_df.csv')
player_weights_df = pd.read_csv('player_weights.csv')


len(df_2.columns)

41

In [95]:
profile_df = pd.concat([df_1, df_2])
profile_df = pd.concat([profile_df, df_3])

In [107]:
# pd. set_option('display.max_rows', 5000)

# profile_df['Date of birth:'] = profile_df['Date of birth:'].fillna('') 
# profile_df[profile_df['Date of birth:'].str.contains('Executive assist...')]


# df = pd.DataFrame(profile_df['club:'].value_counts()).reset_index()['index']
# list(df)
# pd.DataFrame(df).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/jobs.csv', index=False)

In [108]:
# profile_df[profile_df['Place of birth:'].str.contains('Head of first')]
# profile_df.loc[profile_df['Place of birth:'].fillna('').str.contains("Head of first", case=False)]

#  'Head of first te...'
#  'Director of Foot...'
#  'Member of the Su...'
#  'Chief Executive ...'


In [100]:
# Merge Profile Information with Injuries

profile_injuries_df = pd.merge(left=player_injuries_df, right=profile_df, left_on=['TMId', 'name'], right_on=['TMId', 'name'], how='left')

In [101]:
profile_injuries_df

,Season,Injury,from,until,Days,Games missed,TMId,name,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55211,20/21,Sprained ankle,"May 4, 2021","Jun 9, 2021",36 days,2,159907,Jonathan Iglesias,Montevideo,Uruguay France,"1,70 m",midfield - Attacking Midfield,"Jun 30, 2022",NaN,NaN,First Tier,"Jan 30, 2017","Jun 30, 2022",NaN,NaN,NaN,NaN,"Dec 17, 1988",32,right,NaN,Clermont Foot 63,NaN,NaN,Jonathan Damián Iglesias Abreu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55212,20/21,Groin Injury,"Jan 28, 2021","Feb 13, 2021",16 days,4,149579,Elbasan Rashani,Hillerstorp,Kosovo Norway,"1,81 m",attack - Left Winger,"Jun 30, 2023",NaN,20/3,First Tier,"Jul 16, 2021","Jun 30, 2023",NaN,NaN,NaN,NaN,"May 9, 1993",28,left,NaN,Clermont Foot 63,NaN,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55213,19/20,Groin Injury,"Jun 15, 2020","Jun 22, 2020",7 days,2,149579,Elbasan Rashani,Hillerstorp,Kosovo Norway,"1,81 m",attack - Left Winger,"Jun 30, 2023",NaN,20/3,First Tier,"Jul 16, 2021","Jun 30, 2023",NaN,NaN,NaN,NaN,"May 9, 1993",28,left,NaN,Clermont Foot 63,NaN,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55214,15/16,Groin Injury,"Nov 14, 2015","Dec 31, 2015",47 days,3,149579,Elbasan Rashani,Hillerstorp,Kosovo Norway,"1,81 m",attack - Left Winger,"Jun 30, 2023",NaN,20/3,First Tier,"Jul 16, 2021","Jun 30, 2023",NaN,NaN,NaN,NaN,"May 9, 1993",28,left,NaN,Clermont Foot 63,NaN,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
TMId_df = profile_injuries_df[['name', 'TMId']]
TMId_df.drop_duplicates(inplace=True)
TMId_df.sort_values(by=['name'], inplace=True)
TMId_df.reset_index(inplace=True)
TMId_df.drop(columns=['index'], inplace=True)
TMId_df



<ipython-input-66-53e38306e5dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TMId_df.drop_duplicates(inplace=True)
<ipython-input-66-53e38306e5dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TMId_df.sort_values(by=['name'], inplace=True)
/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,name,TMId
0,Aaron Boupendza,427605
1,Aaron Connolly,434207
2,Aaron Cresswell,92571
3,Aaron Doran,96148
4,Aaron Herzog,276566
...,...,...
9186,Óscar de Marcos,96718
9187,Ömer Toprak,43512
9188,Örjan Nyland,73517
9189,Özkan Yildirim,83256


In [67]:
player_weights_df.sort_values(by=['Name', 'year'], inplace=True)
player_weights_df.reset_index(inplace = True)
player_weights_df.drop(columns=['index'], inplace=True)
player_weights_df

# messi = player_weights_df[player_weights_df['Name']=='L. Messi']

# messi.loc[messi['year'].str[-2:].astype(int) - 1 < 10, 'season'] = '0' + (messi['year'].str[-2:].astype(int) - 1).astype(str) + '/' + (messi['year'].str[-2:])
# messi.loc[messi['year'].str[-2:].astype(int) - 1 >= 10, 'season'] = (messi['year'].str[-2:].astype(int) - 1).astype(str) + '/' + (messi['year'].str[-2:])
# messi['Surname'] = messi['Name'].str.split().str[-1]


# messi.bfill(axis=0)

,Name,Age,Overall Rating,Potential,Team,Contract expiry,Height,Weight,Strong foot,Value,Full Name,Nationality,date,year
0,A. Abdennour,19.0,66.0,73.0,SV Werder Bremen,2008,187cm,84kg,Left,€0,Aymen Abdennour,Tunisia,"Feb 22, 2010",FIFA 10
1,A. Abdennour,21.0,74.0,78.0,Toulouse Football Club,2011 ~ 2016,187cm,84kg,Left,€4.2M,Aymen Abdennour,Tunisia,"Feb 22, 2012",FIFA 12
2,A. Abdennour,22.0,78.0,85.0,Toulouse Football Club,2011 ~ 2017,187cm,84kg,Left,€11M,Aymen Abdennour,Tunisia,"Sep 20, 2013",FIFA 13
3,A. Abdennour,23.0,81.0,85.0,AS Monaco,2014 ~ 2016,187cm,85kg,Left,€9M,Aymen Abdennour,Tunisia,"Sep 19, 2014",FIFA 14
4,A. Abdennour,24.0,78.0,80.0,AS Monaco,2014 ~ 2018,187cm,84kg,Left,€7.5M,Aymen Abdennour,Tunisia,"Sep 10, 2015",FIFA 15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fifaleague.it,NaN,"Feb 22, 2007",FIFA 07
38370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sign in with Facebook,NaN,"Feb 22, 2007",FIFA 07
38371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sign in with Twitter,NaN,"Feb 22, 2007",FIFA 07
38372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sign in with Disqus,NaN,"Feb 22, 2007",FIFA 07


In [109]:
profile_weights_TMId_df = pd.merge(left=player_weights_df, right=TMId_df, left_on=['Full Name'], right_on=['name'], how='left')
print(len(profile_injuries_df))
print(len(profile_weights_TMId_df))

55216
38652


In [104]:
TMId_df[TMId_df['name'].str.contains('Alonso')]

,name,TMId
271,Alejandro Alonso,33935
3706,Iván Alonso,51583
4381,José Alonso Lara,461858
5646,Marcos Alonso,112515
8917,Xabi Alonso,7476


In [110]:
# print(len(profile_weights_TMId_df))
# profile_weights_TMId_df['TMId'].value_counts(dropna=False)

NaN_names_df = profile_weights_TMId_df[profile_weights_TMId_df['TMId'].isna()]
NaN_names_df.reset_index(inplace = True)
NaN_names_df.drop(columns = ['index'], inplace=True)

len(NaN_names_df)



/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


15555

In [105]:
for i in range(len(NaN_names_df[:100])):
    for row in range(len(TMId_df)):
        if TMId_df.loc[row, 'name'] in NaN_names_df.loc[i, 'Full Name']:
            NaN_names_df.loc[i, 'TMId'] = TMId_df[row, 'TMId']
        else:
            continue

KeyError: (673, 'TMId')

In [88]:
NaN_names_df['TMId'].value_counts()

Series([], Name: TMId, dtype: int64)

In [40]:
profile_injuries_df[profile_injuries_df['name_x'] == 'Lionel Messi']

,Season,Injury,from,until,Days,Games missed,TMId,name_x,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Retired since:,club:,Name in home country:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Outfitter:,Social-Media:,Agent:,Without Club since:,Full name:,Contract option:,National player:,Current international:,country:,Career break since:,On loan from:,Contract there expires:,Unknown since:,Date of death:,2nd club:,3nd club:,Company:,Ban since:
32691,21/22,Knee Problems,"Oct 29, 2021","Nov 11, 2021",13 days,3,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32692,21/22,Bone Bruise,"Sep 20, 2021","Sep 27, 2021",7 days,2,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32693,21/22,Fitness,"Aug 1, 2021","Aug 26, 2021",25 days,4,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32694,20/21,Hamstring Injury,"Jan 12, 2021","Jan 15, 2021",3 days,1,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32695,20/21,Ankle Injury,"Dec 23, 2020","Jan 1, 2021",9 days,1,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32696,19/20,Adductor problems,"Sep 25, 2019","Oct 1, 2019",6 days,1,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32697,19/20,Foot Injury,"Aug 5, 2019","Sep 16, 2019",42 days,4,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32698,18/19,Pubitis,"Mar 23, 2019","Mar 29, 2019",6 days,-,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32699,18/19,Fore